In [4]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import math

# Import the backtrader platform
import backtrader as bt

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# GBR

size = [1,5,10,20]
lag = [22,63,126,252]

In [57]:
# Create a Stratey
class GBRStrategy(bt.Strategy):
    
    
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.dataclose_spx = self.datas[1].close

        
        # Add indicators
        self.sma_short = bt.indicators.SimpleMovingAverage(self.datas[0], period=50)
        self.sma_long = bt.indicators.SimpleMovingAverage(self.datas[0], period=200)
        
        # Add X variables
        self.dsma = self.sma_short - self.sma_long
        self.mom = bt.Cmp(self.sma_short, self.sma_long)
        
        self.rsi_overbought = bt.indicators.RSI(self.datas[0]) > 70
        self.rsi_oversold = bt.indicators.RSI(self.datas[0]) <30
        
        self.cross_up = bt.indicators.CrossUp(self.sma_short, self.sma_long) 
        self.cross_down = bt.indicators.CrossDown(self.sma_short, self.sma_long)
        
        # Compute daily return
        self.vix_returns = self.dataclose / self.dataclose(-1) - 1
        

    def next(self):
        
        lag = 1
        # Set the length of training set
        trainsize = 63
        count = len(self.vix_returns) - 200
        
        # Simply log the closing price of the series from the reference

        #self.log('VIX Return, %.4f' % self.vix_returns[0])
        
        # Regression
        
        trainX1 = pd.DataFrame(self.mom.get(ago=-lag, size=trainsize).tolist())
        trainX2 = pd.DataFrame(self.rsi_overbought.get(ago=-lag, size=trainsize).tolist())
        trainX3 = pd.DataFrame(self.rsi_oversold.get(ago=-lag, size=trainsize).tolist())
        trainX4 = pd.DataFrame(self.cross_up.get(ago=-lag, size=trainsize).tolist())
        trainX5 = pd.DataFrame(self.cross_down.get(ago=-lag, size=trainsize).tolist())
        
        trainX = pd.concat([trainX1, trainX2, trainX3, trainX4, trainX5], axis=1)
        
        trainY = np.array(self.vix_returns.get(ago=0, size=trainsize).tolist()).reshape(-1, 1)
        
        testX = np.array([self.mom[1-lag], self.rsi_overbought[1-lag], self.rsi_oversold[1-lag], self.cross_up[1-lag], self.cross_down[1-lag]]).reshape(-1,5)
        testY = self.vix_returns[0]
        
        mse = []
        r_pred_gb = []
        y_pred = []
        
        
        if count <= trainsize + lag - 1:
            
            # Set the predicted return to 0 if no regression is running
            r_pred_gb.append(0)
            
        else:
            
            gb_regr = GradientBoostingRegressor()
            gb_regr.fit(trainX, trainY.ravel())
            tmp_pred = gb_regr.predict(testX)
            y_pred = gb_regr.predict(trainX)
            
            r_pred_gb.append(tmp_pred) # use current data to predict
            mse.append(math.pow((tmp_pred-testY), 2))
           # print('%.4f' % r2_score(trainY,y_pred))
            print('%.4f' % mse[-1])
            
            
        #print('Predicted return, %.4f' % r_pred_gb[-1])
        
         
        if r_pred_gb[-1] > 0:

                # BUY, BUY, BUY!!! (with all possible default parameters)
           # self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
            self.order = self.buy()

        if r_pred_gb[-1] < 0:
                # SELL, SELL, SELL!!! (with all possible default parameters)
           # self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
            self.order = self.sell()

In [58]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(GBRStrategy)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mySharpe')

    datapath_spx = os.path.join('../../../datas/spx-2013-2018.txt')
    datapath_vix= os.path.join('../../../datas/vix-2013-2018.txt')

    # Create a Data Feed
    data_vix = bt.feeds.YahooFinanceCSVData(
        dataname=datapath_vix,
        # Do not pass values before this date
        #fromdate=datetime.datetime(2018, 1, 1),
        # Do not pass values before this date
        #todate=datetime.datetime(2018, 2, 9),
        # Do not pass values after this date
        reverse=False)

    data_spx = bt.feeds.YahooFinanceCSVData(
        dataname=datapath_spx,
        # Do not pass values before this date
        #fromdate=datetime.datetime(2018, 1, 1),
        # Do not pass values before this date
        #todate=datetime.datetime(2018, 2, 9),
        # Do not pass values after this date
        reverse=False)

    
    # Add the Data Feed to Cerebro
    cerebro.adddata(data_vix)
    cerebro.adddata(data_spx)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Write output
    cerebro.addwriter(bt.WriterFile, out='gbr_252_1.csv',csv=True)
    
    # Print out the starting conditions
    #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    thestrats = cerebro.run()
    thestrat = thestrats[0]

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    #print('Holding Period Return:', thestrat.analyzers.myHPY.get_analysis())
    print('Sharpe Ratio:', thestrat.analyzers.mySharpe.get_analysis())

0.0005
0.0002
0.0170
0.0163
0.0005
0.0002
0.0002
0.0000
0.0011
0.0004
0.0161
0.0093
0.0134
0.0037
0.0024
0.0009
0.0016
0.0002
0.0037
0.0051
0.0002
0.0001
0.0008
0.0021
0.0001
0.0009
0.0026
0.0139
0.0016
0.0043
0.0229
0.0050
0.0028
0.0009
0.0076
0.0027
0.0000
0.0000
0.0001
0.0001
0.0035
0.0000
0.0002
0.0003
0.0001
0.0004
0.0011
0.0017
0.0001
0.0001
0.0016
0.0025
0.0000
0.0000
0.0063
0.0030
0.0000
0.0018
0.0061
0.0002
0.0026
0.0003
0.0003
0.0000
0.0001
0.0006
0.0008
0.0004
0.0008
0.0055
0.0020
0.0000
0.0037
0.0071
0.0008
0.0016
0.0019
0.0128
0.0001
0.0007
0.0003
0.0114
0.0017
0.0000
0.0008
0.0009
0.0011
0.0006
0.0017
0.0102
0.0035
0.0006
0.0065
0.0016
0.0004
0.0002
0.0060
0.1012
0.0292
0.0037
0.0020
0.0034
0.0008
0.0050
0.0001
0.0031
0.0000
0.0702
0.0000
0.0136
0.0119
0.0013
0.0001
0.0035
0.0106
0.0001
0.0082
0.0015
0.0030
0.0043
0.0001
0.0014
0.0000
0.0008
0.0003
0.0001
0.0001
0.0004
0.0000
0.0001
0.0000
0.0003
0.0023
0.0018
0.0036
0.0025
0.0001
0.0013
0.0032
0.0104
0.0001
0.0029
0.0000

Gradient Boosting Regressor (Lag = 1):

Mean Squared Error, 0.1143

Final Portfolio Value: 109367.76


Lag = 5:

Mean Squared Error, 0.0051

Final Portfolio Value: 107887.95


Lag = 10:

Mean Squared Error, 0.0202

Final Portfolio Value: 105910.57


Lag = 20:

Mean Squared Error, 0.0201

Final Portfolio Value: 107116.08